In [47]:
# -------------------------
# 1. IMPORT REQUIRED LIBRARIES
# -------------------------
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
#from tensorflow.keras.optimizers import Adam


In [48]:
# -------------------------
# 2. SET PATHS
# -------------------------
train_dir = r"C:\Users\\OneDrive\Desktop\DL\DLDataset\ObjDet\train"
test_dir = r"C:\Users\\OneDrive\Desktop\DL\DLDataset\ObjDet\test"
weights_path = r"C:\Users\\OneDrive\Desktop\DL\DLDataset\ObjectDetectionAssn6\vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"

In [49]:
# -------------------------
# 3. LOAD AND PREPROCESS DATA
# -------------------------
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical'
)

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical'
)


Found 176 images belonging to 3 classes.
Found 42 images belonging to 3 classes.


In [50]:
# -------------------------
# 4. LOAD PRETRAINED VGG16 MODEL (without top layers)
# -------------------------
base_model = VGG16(
    weights=None,  # we’ll load custom weights manually
    include_top=False,
    input_shape=(224, 224, 3)
)
base_model.load_weights(weights_path)


In [51]:
# -------------------------
# 5. FREEZE LOWER LAYERS
# -------------------------
for layer in base_model.layers:
    layer.trainable = False

# -------------------------
# 6. ADD CUSTOM CLASSIFIER
# -------------------------
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_data.num_classes, activation='softmax')
])

In [52]:
# -------------------------
# 7. COMPILE MODEL
# -------------------------
model.compile(optimizer= 'Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# -------------------------
# 8. TRAIN CLASSIFIER LAYERS
# -------------------------
history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=5
)


Epoch 1/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 43s 4s/step - accuracy: 0.7330 - loss: 1.8045 - val_accuracy: 0.9524 - val_loss: 0.1721
Epoch 2/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 41s 4s/step - accuracy: 0.9716 - loss: 0.0943 - val_accuracy: 1.0000 - val_loss: 3.5657e-04
Epoch 3/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 45s 4s/step - accuracy: 0.9773 - loss: 0.0820 - val_accuracy: 1.0000 - val_loss: 1.4700e-05
Epoch 4/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 38s 3s/step - accuracy: 0.9830 - loss: 0.0556 - val_accuracy: 0.9762 - val_loss: 0.0263
Epoch 5/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 39s 4s/step - accuracy: 0.9943 - loss: 0.0058 - val_accuracy: 1.0000 - val_loss: 0.0068


In [53]:
# -------------------------
# 9. FINE-TUNE (OPTIONAL)
# -------------------------
# Unfreeze last few convolutional layers for fine-tuning
for layer in base_model.layers[-4:]:
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

fine_tune_history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=1
)

# -------------------------
# 10. EVALUATE MODEL
# -------------------------
loss, acc = model.evaluate(test_data)
print(f"\n✅ Test Accuracy: {acc:.2f}")

11/11 ━━━━━━━━━━━━━━━━━━━━ 53s 4s/step - accuracy: 1.0000 - loss: 2.7146e-04 - val_accuracy: 1.0000 - val_loss: 1.4051e-05
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 1.0000 - loss: 1.4051e-05

✅ Test Accuracy: 1.00


In [33]:
# -------------------------
# 11. SAVE MODEL WEIGHTS
# -------------------------
model.save_weights("fine_tuned_vgg16.weights.h5")
print("\nModel weights saved successfully!")

# -------------------------
# 12. MAKE SAMPLE PREDICTIONS
# -------------------------
import numpy as np
sample_batch, labels = next(test_data)
predictions = np.argmax(model.predict(sample_batch), axis=1)
print("\nPredicted Class IDs:", predictions)



Model weights saved successfully!
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step

Predicted Class IDs: [2 0 0 1 0 0 0 0 2 1 2 0 0 0 2 1]
